In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
%matplotlib inline

## From Nixon & King 2012
### $\partial_t\vec{L} =\,...$

### term 1: $...\,\frac{1}{r}\partial_r\left\{\frac{\partial_r\left[\nu_1\Sigma r^3(-\Omega')\right]}{\Sigma\partial_r(r^2\Omega)}\vec{L}\right\}$

### term 2: $...\,+\,\frac{1}{r}\partial_r\left[\frac{1}{2}\nu_2r|\vec{L}|\partial_r\vec{l}\right]$

### term 3: $...\,+\,\frac{1}{r}\partial_r\left\{\left[\frac{\frac{1}{2}\nu_2r^3\Omega|\partial_r\vec{l}|^2}{\partial_r(r^2\Omega)} + \nu_1(\frac{r\Omega'}{\Omega})\right]\vec{L}\right\} = \frac{1}{r}\partial_r\left\{\left[\nu_2r^2(\partial_r\vec{l})^2 - \frac{3}{2}\nu_1\right]\vec{L}\right\}$

### term 4: $...\,+\,\frac{1}{r}\partial_r\left\{\nu_3r\vec{L}\times\partial_r\vec{l}\right\}$

### term 5: $...\,+\,\vec{\Omega}_p\times\vec{L}$

### And now we want to discretize this, so for convenience, lets just try it term by term....
### Central difference scheme for terms 1, 2, and 4; upwind scheme for 3 (because this is an advective term and the rest are diffusive)

### $\vec{L}^{n+1}_i = \vec{L}^n_i + \Delta t(\vec{f}_{1,i} + \vec{f}_{2,i} + \vec{f}_{3,i} + \vec{f}_{4,i} + \vec{f}_{5,i})$

### $\vec{f}_{1,i} \approx -\frac{1}{r_i\Delta r_i}\left\{\frac{(\nu_{1,i+1}\Sigma_{i+1}r^3_{i+1} + \nu_{1,i}\Sigma_{i}r^3_{i})\frac{1}{\Delta r_{i+1} + \Delta r_i}(\Omega_{i+1}-\Omega_{i}) - \nu_{1,i}\Sigma_i r^3_i\frac{1}{2\Delta r_i}(\Omega_{i+1} - \Omega_{i-1})}{(\frac{1}{2}\Sigma_{i+1} + \frac{3}{2}\Sigma_i)(r^2_{i+1}\Omega_{i+1} - r^2_i\Omega_i)}(3\vec{L}_i + \vec{L}_{i+1}) - \frac{\nu_{1,i}\Sigma_i r^3_i\frac{1}{2\Delta r_i}(\Omega_{i+1} - \Omega_{i-1}) - (\nu_{1,i}\Sigma_{i}r^3_{i} + \nu_{1,i-1}\Sigma_{i-1}r^3_{i-1})\frac{1}{\Delta r_{i} + \Delta r_{i-1}}(\Omega_{i}-\Omega_{i-1})}{(\frac{1}{2}\Sigma_{i} + \frac{3}{2}\Sigma_{i-1})(r^2_i\Omega_i - r^2_{i-1}\Omega_{i-1})}(3\vec{L}_i + \vec{L}_{i-1})\right\}$

### $\vec{f}_{2,i} \approx \frac{1}{16r_i}\frac{1}{\Delta r^2}\left[(\nu_{2,i+1}+\nu_{2,i})(r_{i+1} + r_{i})(|\vec{L}|_{i+1} + |\vec{L}|_{i})(\vec{l}_{i+1} - \vec{l}_{i}) - (\nu_{2,i}+\nu_{2,i-1})(r_{i} + r_{i-1})(|\vec{L}|_{i} + |\vec{L}|_{i-1})(\vec{l}_{i} - \vec{l}_{i-1})\right]$

### We will use upwind differencing for the third term because it is advective
### The following uses forward differencing and is valid if $V_{\rm adv} < 0$, as it indicates the advected material is coming from $r_{i+1} \rightarrow r_{i}$. In the case that $V_{\rm adv} > 0$, the equation can be amended by multiplying by $-1$ and taking $i+1\rightarrow i$ and $i \rightarrow i-1$
### We define the advective velocity as: 
### $v_{\rm adv} = \frac{\nu_2|\psi|^2}{r} - \frac{3\nu_1}{2r}$

### $\vec{f}_{3,i} \approx \frac{1}{r_i\Delta r_i}\left\{\left[\frac{(\Delta r_{i+1} + \Delta r_i)(\nu_{2,i+1}r_{i+1}\Omega_{i+1}|\psi|^2_{i+1} + \nu_{2,i}r_{i}\Omega_{i}|\psi|^2_{i})}{4(r_{i+1}^2\Omega_{i+1}^2 - r_i^2\Omega_i^2)} + \frac{1}{2}(\frac{\nu_{1,i+1}r_{i+1}}{\Omega_{i+1}} + \frac{\nu_{1,i}r_i}{\Omega_i})\frac{1}{\Delta r_{i+1} + \Delta r_i}(\Omega_{i+1} - \Omega_i)\right]\frac{1}{2}(\vec{L}_{i+1} + \vec{L}_{i}) - \left[\frac{\Delta r_i\nu_{2,i}r_i\Omega_i|\psi|^2_i}{r^2_{i+1}\Omega_{i+1} - r^2_i\Omega_i} + \frac{\nu_{1,i}r_i}{2\Omega_i\Delta r_i}(\Omega_{i+1} - \Omega_i)\right]\vec{L}_i\right\}$

### We will write $\vec{f}_{4,i}$ in terms of its $x$, $y$ and $z$ components because of the cross product

### $(\vec{f}_{4,i})_x \approx \frac{1}{8r_i}\frac{1}{\Delta r_i^2}\left\{(\nu_{3,i+1}+\nu_{3,i})(r_{i+1}+r_{i})\left[(\vec{L}_{i+1}+\vec{L}_{i})_y(\vec{l}_{i+1}-\vec{l}_{i})_z - (\vec{L}_{i+1}+\vec{L}_{i})_z(\vec{l}_{i+1}-\vec{l}_{i})_y\right] - (\nu_{3,i}+\nu_{3,i-1})(r_{i}+r_{i-1})\left[(\vec{L}_{i}+\vec{L}_{i-1})_y(\vec{l}_{i}-\vec{l}_{i-1})_z - (\vec{L}_{i}+\vec{L}_{i-1})_z(\vec{l}_{i}-\vec{l}_{i-1})_y\right]\right\}$

### $(\vec{f}_{4,i})_y \approx \frac{1}{8r_i}\frac{1}{\Delta r_i^2}\left\{(\nu_{3,i+1}+\nu_{3,i})(r_{i+1}+r_{i})\left[(\vec{L}_{i+1}+\vec{L}_{i})_z(\vec{l}_{i+1}-\vec{l}_{i})_x - (\vec{L}_{i+1}+\vec{L}_{i})_x(\vec{l}_{i+1}-\vec{l}_{i})_z\right] - (\nu_{3,i}+\nu_{3,i-1})(r_{i}+r_{i-1})\left[(\vec{L}_{i}+\vec{L}_{i-1})_z(\vec{l}_{i}-\vec{l}_{i-1})_x - (\vec{L}_{i}+\vec{L}_{i-1})_x(\vec{l}_{i}-\vec{l}_{i-1})_z\right]\right\}$

### $(\vec{f}_{4,i})_z \approx \frac{1}{8r_i}\frac{1}{\Delta r_i^2}\left\{(\nu_{3,i+1}+\nu_{3,i})(r_{i+1}+r_{i})\left[(\vec{L}_{i+1}+\vec{L}_{i})_x(\vec{l}_{i+1}-\vec{l}_{i})_y - (\vec{L}_{i+1}+\vec{L}_{i})_y(\vec{l}_{i+1}-\vec{l}_{i})_x\right] - (\nu_{3,i}+\nu_{3,i-1})(r_{i}+r_{i-1})\left[(\vec{L}_{i}+\vec{L}_{i-1})_x(\vec{l}_{i}-\vec{l}_{i-1})_y - (\vec{L}_{i}+\vec{L}_{i-1})_y(\vec{l}_{i}-\vec{l}_{i-1})_x\right]\right\}$

### $\vec{f}_{5,i} = (\Omega_p)_i(-(\vec{L}_i)_{y}\hat{x} + (\vec{L}_i)_{x}\hat{y})$